# Section 1
#### Import necessary packages for feature engineering

In [1]:
import pandas as pd
import numpy as np

In [4]:
from feature_engine.encoding import RareLabelEncoder
from feature_engine.encoding import OneHotEncoder
from feature_engine.imputation import CategoricalImputer
from feature_engine.imputation import MeanMedianImputer
from feature_engine.transformation import LogTransformer
from feature_engine.transformation import YeoJohnsonTransformer
from feature_engine.selection import DropConstantFeatures

#### Load cleaned dataset

In [5]:
df = pd.read_csv('/Users/jotech/Milestone Repo/milestone-project-market-prediction-model/data/FET-USD-CLEANED.csv')
df.head()

,Date,Open,High,Low,Close,Adj Close,Volume
0,2023-06-21,0.199379,0.227389,0.199379,0.223477,0.223477,66362316
1,2023-06-20,0.189383,0.199555,0.182674,0.199320,0.199320,32274029
2,2023-06-19,0.183223,0.192260,0.181794,0.189395,0.189395,28179454
3,2023-06-18,0.197603,0.197603,0.182670,0.183230,0.183230,28836777
4,2023-06-17,0.190300,0.199963,0.189429,0.197628,0.197628,24247119


In [6]:
constant_features = DropConstantFeatures(tol = 0.998)
constant_features.fit(df)

DropConstantFeatures(tol=0.998)